In [3]:
# author: Jan Tschada
# SPDX-License-Identifer: Apache-2.0

In [18]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from arcgis.geometry import union
from arcgis.geometry.filters import intersects
from arcgis.mapping import generate_classbreaks
from datetime import datetime
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from georapid.fires import aggregate
from georapid.formats import OutFormat

## Creating a client

**NOTE**

Ensure that `os.environ['x_rapidapi_key']` holds your Rapid API Key!


In [2]:
host = 'geofires.p.rapidapi.com'
client: GeoRapidClient = EnvironmentClientFactory.create_client_with_host(host)

In [8]:
date_of_interest = datetime(2024, 1, 8)
aggregated_fires_dict = aggregate(client, date_of_interest, OutFormat.ESRI)
aggregated_fires_fset = FeatureSet.from_dict(aggregated_fires_dict)
aggregated_fires_sdf = aggregated_fires_fset.sdf
aggregated_fires_sdf

,index,count,OBJECTID,SHAPE
0,0,24,1,"{""rings"": [[[-8995684.444548402, 5537508.34279..."
1,1,39,2,"{""rings"": [[[-12459786.059686158, 7787508.3427..."
2,2,40,3,"{""rings"": [[[-8562671.742656182, 5787508.34279..."
3,3,17,4,"{""rings"": [[[-12892798.761578377, 7537508.3427..."
4,8,31,5,"{""rings"": [[[-8129659.040763963, 6037508.34279..."
...,...,...,...,...
279,928,1,280,"{""rings"": [[[314088.6461343169, 6662508.342799..."
280,942,1,281,"{""rings"": [[[-8129659.040763963, -3962491.6572..."
281,943,1,282,"{""rings"": [[[-13758824.165362816, 4787508.3427..."
282,992,1,283,"{""rings"": [[[963607.6989726461, 8287508.342799..."


## Map news related to wildfires in California

In [9]:
def generate_aggregated_renderer(spatial_df, column='count'):
    alpha = 0.35
    class_breaks_renderer = generate_classbreaks(spatial_df, geometry_type='Polygon', field=column, method='esriClassifyNaturalBreaks', class_count=5, colors='YlOrRd', alpha=0.35)
    class_breaks_renderer['defaultSymbol']['color'][3] = int(alpha * 255)
    for class_break_info in class_breaks_renderer['classBreakInfos']:
        class_break_info['symbol']['color'][3] = int(alpha * 255)
    return class_breaks_renderer

def plot_aggregated(map_view, spatial_df, column='count'):
    """
    Plots the spatial dataframe as classified polygons using the specified map view.
    """
    if spatial_df.empty:
        print("The dataframe is empty!")
    else:
        class_breaks_renderer = generate_aggregated_renderer(spatial_df)
        drawing_info = {
            'renderer': class_breaks_renderer
        }
        feature_collection = spatial_df.spatial.to_feature_collection(drawing_info=drawing_info)
        map_view.add_layer(feature_collection)

In [10]:
gis = GIS()

In [11]:
map_view = gis.map('California, USA')
map_view.basemap = 'osm'

In [12]:
plot_aggregated(map_view, aggregated_fires_sdf)
map_view

MapView(layout=Layout(height='400px', width='100%'))

In [13]:
item_id = 'b8f4033069f141729ffb298b7418b653'
thermal_item = gis.content.get(item_id)
thermal_item

<Item title:"Satellite (MODIS) Thermal Hotspots and Fire Activity" type:Feature Layer Collection owner:esri_livefeeds2>

In [30]:
aggregated_fires_union = union(aggregated_fires_sdf[aggregated_fires_sdf['count'] > 20].SHAPE.tolist())
spatial_filter = intersects(aggregated_fires_union[0])

In [31]:
thermal_layer = thermal_item.layers[0]
thermal_events_sdf = thermal_layer.query(geometry_filter=spatial_filter, as_df=True)
thermal_events_sdf

,OBJECTID,BRIGHTNESS,SCAN,TRACK,SATELLITE,CONFIDENCE,VERSION,BRIGHT_T31,FRP,ACQ_DATE,DAYNIGHT,HOURS_OLD,DAY_OF_ACQ,SHAPE
0,48125,363.83,3.53,1.77,A,94,6.1NRT,294.16,608.38,2025-01-07 22:39:00,D,95,4,"{""x"": -13199346.8324, ""y"": 4033732.3479999974,..."
1,48126,325.93,3.57,1.77,A,75,6.1NRT,291.73,136.96,2025-01-07 22:39:00,D,95,4,"{""x"": -13191393.0548, ""y"": 4036459.647500001, ..."
2,48127,425.19,3.55,1.77,A,87,6.1NRT,302.8,2749.04,2025-01-07 22:39:00,D,95,4,"{""x"": -13195638.7802, ""y"": 4036131.7995999977,..."
3,48128,440.34,3.53,1.77,A,75,6.1NRT,310.73,3688.14,2025-01-07 22:39:00,D,95,4,"{""x"": -13199877.8265, ""y"": 4035801.2739999965,..."
4,48129,310.43,3.51,1.76,A,0,6.1NRT,293.04,46.99,2025-01-07 22:39:00,D,95,4,"{""x"": -13204217.0602, ""y"": 4035461.3532000035,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,100493,362.77,4.08,1.87,T,100,6.1NRT,293.86,711.13,2025-01-11 17:04:00,D,4,0,"{""x"": -13191456.507, ""y"": 4045926.9679000005, ..."
424,100494,374.92,4.08,1.87,T,100,6.1NRT,296.78,1018.28,2025-01-11 17:04:00,D,4,0,"{""x"": -13191664.6744, ""y"": 4043698.8915000036,..."
425,100495,327.68,4.11,1.88,T,84,6.1NRT,290.47,184.11,2025-01-11 17:04:00,D,4,0,"{""x"": -13196459.2048, ""y"": 4041760.2688999996,..."
426,100496,410.03,4.08,1.87,T,100,6.1NRT,299.7,2423.56,2025-01-11 17:04:00,D,4,0,"{""x"": -13191882.8606, ""y"": 4041471.251500003, ..."


In [33]:
thermal_events_sdf.spatial.plot(map_view)
map_view

MapView(jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=True)